### Transformer Together

👋 Thanks for agreeing to join our wierd tutorial-o-thon. Here're the things that you want to know:

#### Participants

- @av
- @stradivar
- @RomanSavarin
- @wpierwolacraft
- @StanislavRassolenko

#### Goal

- Go through as large chunk of [Karpathy's NanoGPT tutorial](https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy) as we can
- Understand all the relevants bits and pieces along the way the best we can
- Nerd out
- Enjoy the beautiful awrkwardness of trying to code something together with a few random dudes

We'll likely won't complete the tutorial, but we'll try to tackle as much as possible


#### Logistics

- Arrive around 12am, May 26th, the address was sent to you privately
- @av will order pizza and provide snacks/drinks, but feel free to bring your own
- Have your laptop with you, maybe a charger, Internet will be available via WiFi
- You'll need a Jupyter-notebook compatible environment, preferrably with a GPU support
  - If you know what you're doing - skip this section, 
  - Otherwise, an easy option is to use [Google Colab](https://colab.research.google.com/)
    - You don't need anything other than Browser and Google Account to start

##### Google Colab setup

Colab allows you to clone arbitrary Notebooks from the GitHub, such as this one 🎉.